In [1]:
import pandas as pd
import re
# import ace_tools as tools

# Raw merged table text
raw = """中文名\t英文名\t平均重量（磅）\t单价（美元/磅）\t单条价格\t难度（1–10）
蓝鳃太阳鱼sohu.com\tBluegillen.wikipedia.org\t~0.5sohu.com\t$13.99galesburgmeatco.com\t~$7galesburgmeatco.comsohu.com\t1fishingbooker.com
瓜仁太阳鱼zh.wikipedia.org\tPumpkinseedzh.wikipedia.org\t~0.5en.wikipedia.org\t$14.0galesburgmeatco.com\t~$7galesburgmeatco.comen.wikipedia.org\t1fishingbooker.com
黑莓鲈fishbase.se\tBlack Crappiegetmyboat.com\t~1.5getmyboat.com\t$20.0walleyedirect.com\t~$30getmyboat.comwalleyedirect.com\t3reddit.com
黄鲈zh.wikipedia.org\tYellow Perchen.wikipedia.org\t~0.5fishbase.se\t$22.99efcmarket.com\t~$11fishbase.seefcmarket.com\t2reddit.com
褐牛鲶en.wikipedia.org\tBrown Bullheaden.wikipedia.org\t~1.1en.wikipedia.org\t$5.0seafarmsva.com\t~$5seafarmsva.comen.wikipedia.org\t2en.wikipedia.org
美洲鳗鲡\tAmerican Eel\t13–17 lb（常见范围）dimensions.com\t$1.25–1.75/lb（黄鳝市场价）seafoodwatch.org\t≈$25（估算）\t4（中等偏易）
鸭嘴鲟\tAmerican Paddlefish\t~60 lb（平均约27 kg）en.wikipedia.org\t~$10/lb（熏制批发价）srac.msstate.edu\t≈$600（估算）\t8（较难）
美洲西鲱\tAmerican Shad\t3–8 lb（常见范围）en.wikipedia.org\t~$1/lb（约2005年市场价）csmonitor.com\t≈$5（估算）\t3（容易）
尖吻鲟\tAtlantic Sturgeon\t常见≤300 lb（最长约15英尺）en.wikipedia.org\t~$300/lb（极稀有，估计）kookssecrets.com\t≥$60,000（估算）\t10（极难）
大口牛胭脂鱼\tBigmouth Buffalo\t2–14 lb（常见范围）a-z-animals.com\t~$1/lb（属低值粗粮鱼）mprnews.org\t≈$10（估算）\t6（中等）
黑牛胭脂鱼\tBlack Buffalo\t≈4.1fishbase.mnhn.fr\t~$5.99samimakolet.com\t~$25（4.1×$5.99）fishbase.mnhn.frsamimakolet.com\t8（肉食性，但很少用钓竿钓获，钓取较难）
黑斑刺盖太阳鱼\tBlack Crappie\t≈0.6fishbase.se\t~$32.50whytesfishery.comwhytesfishery.com\t~$20（0.6×$32.5）fishbase.sewhytesfishery.com\t2（新手易上手facebook.com）
黑鮰\tBlack Bullhead\t≈0.5fishbase.se\t~$2.5（与其他鲶鱼相当）profish.com\t~$1.25（0.5×$2.5）fishbase.seprofish.com\t2（入门级小型鲶鱼，容易钓获）
黑尾沙丁鱼\tBlacktail Shiner\t<0.1（极小，约0.02）\t— （仅作诱饵鱼，不作商品鱼）\t—\t1（个体小，易于用小钩钓获）
蓝鲶鱼\tBlue Catfish\t25–70（典型约50）en.wikipedia.org\t~$2.46profish.com\t~$123（50×$2.46）en.wikipedia.orgprofish.com\t4（中等，大型个体需一定技巧）
长背亚口鱼\tBlue Sucker\t≈5.5（2–3 kg）en.wikipedia.org\t— （极少上市）\t—\t10（难以被钓到）
蓝鳃太阳鱼\tBluegill\t≈0.3fishbase.mnhn.fr\t~$24.95michiganseagrant.org\t~$7.5（0.3×$24.95）fishbase.mnhn.frmichiganseagrant.org\t1（最常见入门鱼种）
弓鳍鱼\tBowfin\t1–5（平均约3）ebsco.com\t— （区域性食用，商品价值低）\t—\t5（攻击性强，咬钩容易但搏斗激烈）
云斑鮰\tBrown Bullhead\t≈0.4fishbase.se\t~$2.5（参考鲶鱼市场价）profish.com\t~$1.0（0.4×$2.5）fishbase.seprofish.com\t2（夜钓常见，容易上钩）
褐鳟\tBrown Trout\t≈2.2en.wikipedia.org\t~$18（参考鲑鳟鱼价）michiganseagrant.org\t~$40（2.2×$18）en.wikipedia.orgmichiganseagrant.org\t7（大型个体警惕性高blueridgeoutdoors.com）
水牛鱼\tBuffalo fish\t3.5\t5.7\t20\t5
链纹狗鱼\tChain Pickerel\t2\t17.95\t35.9\t4
斑点叉尾鮰\tChannel Catfish\t2\t7.25\t14.5\t3
美洲锐唇鲷\tChiselmouth\t2\t3\t6\t6
眼点丽鱼\tButterfly Peacock Bass\t2\t3\t6\t6
美鳊\tCommon Shiner\t0.2\t0.2\t0.04\t2
白马切喉鳟\tCutthroat Trout\t2\t6.54\t13.08\t5
铲鮰\tFlathead Catfish\t2\t7.25\t14.5\t7
佛罗里达雀鳝\tFlorida Gar\t5\t7.25\t36.25\t5
淡水石首鱼\tFreshwater Drum\t3\t3.09\t9.27\t4
金鳟\tGolden Trout\t3\t6.54\t19.62\t5
金体美鳊\tGolden Shiner\t0.2\t0.2\t0.04\t2
绿太阳鱼\tGreen Sunfish\t0.5\t0.2\t0.1\t3
臼齿鱼\tHardhead\t0.5\t0.2\t0.1\t3
双点美鱥\tHornyhead Chub\t0.2\t0.2\t0.04\t3
大口黑鲈\tLargemouth Bass\t2\t17.95\t35.9\t6
北美狗鱼\tMuskellunge\t5\t17.95\t89.75\t9
白斑狗鱼\tNorthern Pike\t4\t17.95\t71.8\t7
七鳃鳗\tPetromyzontidae (Lamprey)\t2\t10\t20\t8
驼背太阳鱼\tPumpkinseed Sunfish\t0.5\t0.2\t0.1\t2
灰西鲱\tAlewife\t0.44fishbase.se\t≈4.0 (马哈鱼替代)\t1.76\t4
虹鳟\tRainbow Trout\t5.0fishbase.se\t≈6.0\t30\t5
美国红鱼\tRed Drum\t25.0fishbase.se\t≈6.0\t150\t7
小冠太阳鱼\tRedear Sunfish\t~0.5\t≈5.0\t2.5\t4
红鳍狗鱼\tRedfin Pickerel\t~2.0\t≈4.0\t8\t6
小太阳鱼\tRedspotted Sunfish\t~0.5\t≈5.0\t2.5\t3
小须美鮠\tRiver Chub\t~0.5\t≈3.0 (鲫鱼类替代)\t1.5\t2
岩钝鲈\tRock Bass\t~3.0fishbase.se\t≈5.0\t15\t5
加拿大梭鲈\tSauger\t~4.0\t≈4.0\t16\t6
短吻鲟\tShortnose Sturgeon\t~4.0fishbase.se\t≈12.0 (白鲟替代)\t48\t9
小口黑鲈\tSmallmouth Bass\t~4.0fishbase.se\t≈5.0\t20\t7
斑点黑鲈\tSpotted Bass\t~3.0fishbase.se\t≈5.0\t15\t6
斑点雀鳝\tSpotted Gar\t~8.0fishbase.mnhn.fr\t≈5.0\t40\t8
硬头鳟\tSteelhead Trout\t~10.0\t≈8.0\t80\t6
美洲条纹狼鲈\tStriped Bass\t~20.0\t≈6.0\t120\t8
丁鱥\tTench\t~4.0fishbase.se\t≈4.0\t16\t5
露仙美鱥\tTennessee Shiner\t~0.5\t≈3.0\t1.5\t3
玻璃梭鲈\tWalleye\t~3.0\t≈8.0\t24\t6
白金火箭\tWhite Alligator Gar\t~5.0\t≈7000.0 (稀有市价)\t35000\t10
白化叉尾鮰\tWhite Channel Catfish\t~5.0\t≈20.0 (观赏品)\t100\t8
白斑刺盖太阳鱼\tWhite Crappie\t~1.0\t≈50.0 (观赏品)\t50\t5
高首鲟\tWhite Sturgeon\t~100.0\t≈15.0\t1500\t9
白化高首鲟\tAlbino White Sturgeon\t~100.0\t≈15000.0 (观赏品)\t1500000\t9
白亚口鱼\tYaqui Sucker\t~2.0\t≈5.0\t10\t4
黄鲈\tYellow Perch\t~0.5\t≈8.0\t4\t4"""

# Parse data
data = []
for line in raw.splitlines()[1:]:
    cols = line.split('\t')
    if len(cols) < 6:
        continue
    ch_raw, en_raw, wt_raw, up_raw, _, diff_raw = cols[:6]
    # Chinese name
    ch = ''.join(re.findall(r'[\u4e00-\u9fff]+', ch_raw))
    # English name
    en0 = en_raw.split('.', 1)[0]
    en = en0[:-2] if en0.endswith(('en','zh')) else en0
    # Weight parsing
    m = re.search(r'(\d+\.?\d*)[–-](\d+\.?\d*)', wt_raw)
    if m:
        wt = (float(m.group(1)) + float(m.group(2))) / 2
    else:
        m2 = re.search(r'(\d+\.?\d*)', wt_raw)
        wt = float(m2.group(1)) if m2 else None
    # Unit price parsing
    m = re.search(r'(\d+\.?\d*)[–-](\d+\.?\d*)', up_raw)
    if m:
        up = (float(m.group(1)) + float(m.group(2))) / 2
    else:
        m2 = re.search(r'(\d+\.?\d*)', up_raw)
        up = float(m2.group(1)) if m2 else None
    # Difficulty parsing
    m = re.search(r'(\d+)', diff_raw)
    diff = int(m.group(1)) if m else None
    data.append({
        '中文名': ch, '英文名': en, 
        '平均重量（磅）': wt, '单价（美元/磅）': up, 
        '难度': diff
    })

df = pd.DataFrame(data)
# Drop duplicate fish by English name
df = df.drop_duplicates(subset='英文名').reset_index(drop=True)

# Normalize for scoring
df['norm_price'] = (df['单价（美元/磅）'] - df['单价（美元/磅）'].min()) / (df['单价（美元/磅）'].max() - df['单价（美元/磅）'].min())
df['norm_diff'] = (df['难度'] - df['难度'].min()) / (df['难度'].max() - df['难度'].min())
df['norm_weight'] = (df['平均重量（磅）'] - df['平均重量（磅）'].min()) / (df['平均重量（磅）'].max() - df['平均重量（磅）'].min())
df['Score'] = 0.5 * df['norm_price'] + 0.3 * df['norm_diff'] + 0.2 * df['norm_weight']

# Assign tiers and sub-tiers
df = df.sort_values('Score', ascending=False).reset_index(drop=True)
tier, subtier = [], []
for i in range(len(df)):
    t = 9 - (i // 7)
    tier.append(t)
    subtier.append(i % 7 + 1)
df['等阶'] = tier
df['细化等阶'] = subtier

# Calculate 单条价格
df['单条价格'] = df['平均重量（磅）'] * df['单价（美元/磅）']

# Prepare final output
out = df[['等阶','细化等阶','中文名','英文名','平均重量（磅）','单价（美元/磅）','单条价格']]

# tools.display_dataframe_to_user("鱼种分阶结果", out)
out


,等阶,细化等阶,中文名,英文名,平均重量（磅）,单价（美元/磅）,单条价格
0,9,1,白化高首鲟,Albino White Sturgeon,100.0,15000.00,1500000.000
1,9,2,白金火箭,White Alligator Gar,5.0,7000.00,35000.000
2,9,3,尖吻鲟,Atlantic Sturgeon,300.0,300.00,90000.000
3,9,4,高首鲟,White Sturgeon,100.0,15.00,1500.000
4,9,5,鸭嘴鲟,American Paddlefish,60.0,10.00,600.000
...,...,...,...,...,...,...,...
57,1,2,瓜仁太阳鱼,Pumpkinseed,0.5,14.00,7.000
58,1,3,蓝鳃太阳鱼,Bluegill,0.5,13.99,6.995
59,1,4,黑尾沙丁鱼,Blacktail Shiner,0.1,NaN,NaN
60,1,5,长背亚口鱼,Blue Sucker,2.5,NaN,NaN


In [ ]:
import os.path
from openpyxl.styles.fonts import Font

rootPath = ".."
OriginWorkbookPath = r"intermediateTables\PreCATFishesOrigin.xlsx"
AffinityWorkbookPath = r"configs\xlsx_channel\2001\fish_env_affinity.xlsx"
FishBasicWorkbookPath = r"configs\xlsx_channel\2001\fish_basic.xlsx"
PondWorkbookPath = r"configs\xlsx_channel\2001\fish_pond.xlsx"
PriceToolWorkbookPath = r"intermediateTables\鱼单价工具.xlsx"

OriginWorkbookPath = os.path.join(rootPath, OriginWorkbookPath)

sheetNameTiers = "鱼种分阶"

# write df to excel
df.to_excel(OriginWorkbookPath, sheet_name=sheetNameTiers, index=False)